In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00


In [19]:
import numpy as np
import pandas as pd

In [13]:
from google.colab import userdata
api = userdata.get('openai')

In [15]:
from openai import OpenAI

client = OpenAI(api_key = api)

In [20]:
data = pd.read_excel('/content/Dataset.xlsx' )

## Drop first 13 columns
data = data.iloc[:, 13:]

data['Purchase Behavior'] = (data['PB1: I am willing to buy a product promoted on social media.']
              + data['PB2: There is a high probability that I would purchase a product because of the impact of social media.']
              + data['PB3: I am easily influenced by advertisements on social media to make impulsive purchases.']
              + data['PB4: I had the experience of buying a product due to the influence of social media.'])/4


data['Purchase Behavior'] = data['Purchase Behavior'].apply(lambda x: 1 if x >= 2.5 else 0)

# Drop PB columns
data = data.iloc[:, 4:]
data.head()

def likert_to_text(value):
  scale = {
      1: "Strongly Disagree",
      2: "Disagree",
      3: "Neutral",
      4: "Agree",
      5: "Strongly Agree"
  }
  return scale.get(value, "Invalid Response")

data.iloc[:, :12] = data.iloc[:, :12].applymap(likert_to_text)

data_drop = data.iloc[:, :8]
data_drop["Purchase Behavior"] = data["Purchase Behavior"]
data_drop.head()

# Split into train and test set
from sklearn.model_selection import train_test_split

X = data_drop.iloc[:, :8]
y = data_drop.iloc[:, 8:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<ipython-input-20-46d8c3e82bbf>:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data.iloc[:, :12] = data.iloc[:, :12].applymap(likert_to_text)


In [21]:
def row_to_question(row):
    return  f"Advertisements on social media help you be aware of products: {row[0]}, " \
            f"Social media ads attract your attention more than other channels: {row[1]}, " \
            f"Keywords like 'promotion' and 'discount' grab your attention: {row[2]}, " \
            f"You have purchased products from social media ads: {row[3]}, " \
            f"Your family influences your purchasing habits: {row[4]}, " \
            f"People around you think you should purchase via social media: {row[5]}, " \
            f"You feel good if many people purchase products via social media: {row[6]}, " \
            f"Your friends encourage you to purchase via social media: {row[7]}, " \
# Apply the function to each row and store the questions

questions_list = X_test.apply(row_to_question, axis=1).tolist()

<ipython-input-21-9a5a5071d8d8>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return  f"Advertisements on social media help you be aware of products: {row[0]}, " \
<ipython-input-21-9a5a5071d8d8>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f"Social media ads attract your attention more than other channels: {row[1]}, " \
<ipython-input-21-9a5a5071d8d8>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f"Keywords like 'promotio

In [47]:
prompt = ". You are an assistant that provides answers to questions based on the previous context. If you think a particular customer is going to make a purchase, say 1. Otherwise say 0. If you can't answer the question, reply \"NaN\". Be as concise as possible and go straight to the point. Do not say anything other than 1 and 0."
messages=[{"role": "user", "content": questions_list[0]+ prompt}]
print(messages)

[{'role': 'user', 'content': 'Advertisements on social media help you be aware of products: Agree, Social media ads attract your attention more than other channels: Neutral, Keywords like \'promotion\' and \'discount\' grab your attention: Strongly Agree, You have purchased products from social media ads: Strongly Agree, Your family influences your purchasing habits: Agree, People around you think you should purchase via social media: Agree, You feel good if many people purchase products via social media: Disagree, Your friends encourage you to purchase via social media: Disagree, . You are an assistant that provides answers to questions based on the previous context. If you think a particular customer is going to make a purchase, say 1. Otherwise say 0. If you can\'t answer the question, reply "NaN". Be as concise as possible and go straight to the point. Do not say anything other than 1 and 0.'}]


In [75]:
responses = []


for question in questions_list:
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": question+prompt}],
        stream=True,
    )

    response = ""  # To hold the full response
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response += chunk.choices[0].delta.content  # Concatenate each token

    responses.append(response)  # Append the full response to y
    print(question)
    print(response)  # Optionally print the full response if needed

Advertisements on social media help you be aware of products: Agree, Social media ads attract your attention more than other channels: Neutral, Keywords like 'promotion' and 'discount' grab your attention: Strongly Agree, You have purchased products from social media ads: Strongly Agree, Your family influences your purchasing habits: Agree, People around you think you should purchase via social media: Agree, You feel good if many people purchase products via social media: Disagree, Your friends encourage you to purchase via social media: Disagree, 
1
Advertisements on social media help you be aware of products: Agree, Social media ads attract your attention more than other channels: Strongly Agree, Keywords like 'promotion' and 'discount' grab your attention: Agree, You have purchased products from social media ads: Agree, Your family influences your purchasing habits: Disagree, People around you think you should purchase via social media: Neutral, You feel good if many people purchase

In [78]:
responses = pd.DataFrame(responses)
responses.shape

(54, 1)

In [85]:
responses = responses.apply(pd.to_numeric, errors='coerce')

numpy.int64

In [88]:
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(y_test, responses)
f1 = f1_score(y_test, responses)

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.65
F1 Score: 0.74
